In [1]:
import cv2
import numpy as np
import librosa
import sounddevice as sd
import queue
import threading
import tensorflow as tf
from face_utils import detect_and_predict_face_emotion
from voice_utils import predict_voice_emotion

In [2]:
face_model = tf.keras.models.load_model('X:/emotion detection/models/face_emotion_model.h5')
voice_model = tf.keras.models.load_model('X:/emotion detection/models/voice_emotion_model.h5')

In [3]:
cap = cv2.VideoCapture(0)


In [4]:
audio_queue = queue.Queue()
voice_emotion = "..."


In [5]:
# Recording config
SAMPLERATE = 22050
CHANNELS = 1
CHUNK_DURATION = 3  # seconds

In [7]:
def audio_recording_thread():
    global voice_emotion
    buffer = []

    def callback(indata, frames, time, status):
        buffer.append(indata.copy())

    with sd.InputStream(samplerate=SAMPLERATE, channels=CHANNELS, callback=callback):
        while True:
            sd.sleep(CHUNK_DURATION * 1000)
            if buffer:
                audio_np = np.concatenate(buffer, axis=0).flatten()
                buffer.clear()
                voice_emotion = predict_voice_emotion(audio_np, voice_model)

# Start recording thread
threading.Thread(target=audio_recording_thread, daemon=True).start()

In [ ]:

# Video Loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    face_emotion = detect_and_predict_face_emotion(frame,face_model)

    # Display emotions
   
    cv2.putText(frame, f"Voice: {voice_emotion}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    cv2.imshow('Multimodal Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 